In [104]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
from talib import abstract
import json


def mean_absolute_percentage_error(actual, prediction):
    actual = pd.Series(actual)
    prediction = pd.Series(prediction)
    return 100 * np.mean(np.abs((actual - prediction))/actual)


def get_arima(data, train_len, test_len):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    train = data.head(train_len).values.tolist()
    test = data.tail(test_len).values.tolist()

    # Initialize model
    model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    prediction = []
    prediction2 = []
    for i in range(len(test)):
        model = pm.ARIMA(order=order)
        model.fit(train)
        print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction.append(model.predict()[0])
        prediction2.append(model.predict()[5])
        train.append(test[i])
#         print(model.predict()[0],test[i],model.predict()[0]-test[i])
    # Generate error data
    mse = mean_squared_error(test, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    
    return prediction,prediction2, mse, rmse, mape


def get_lstm(data, train_len, test_len, lstm_len=4):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    dataset = np.reshape(data.values, (len(data), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset_scaled = scaler.fit_transform(dataset)
    x_train = []
    y_train = []
    x_test = []

    for i in range(lstm_len, train_len):
        x_train.append(dataset_scaled[i - lstm_len:i, 0])
        y_train.append(dataset_scaled[i, 0])
    for i in range(train_len, len(dataset_scaled)):
        x_test.append(dataset_scaled[i - lstm_len:i, 0])

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Set up & fit LSTM RNN
    model = Sequential()
    model.add(LSTM(units=lstm_len, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=int(lstm_len/2)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    model.fit(x_train, y_train, epochs=500, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate predictions
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction).tolist()

    output = []
    for i in range(len(prediction)):
        output.extend(prediction[i])
    prediction = output

    # Generate error data
    mse = mean_squared_error(data.tail(len(prediction)).values, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(data.tail(len(prediction)).reset_index(drop=True), pd.Series(prediction))
    return prediction, mse, rmse, mape




In [ ]:
data = pd.read_csv('SPY_bar_sample_5sec.csv', index_col=0, header=0)
display(data)
data = data.loc['2020-09-04 12:00:00':'2020-09-04 13:00:00'].reset_index(drop=False)


In [187]:
# if __name__ == '__main__':
    # Load historical data
    # CSV should have columns: ['date', 'open', 'high', 'low', 'close', 'volume']
data = pd.read_csv('SPY_bar_sample_5sec.csv', index_col=0, header=0)
display(data)
data = data.loc['2020-09-04 12:00:00':'2020-09-04 13:00:00'].reset_index(drop=False)
# display(data)
data['volume'] = data['volume'].astype('int32')
# del data["average"]
del data["barcount"]

display(data)
# Initialize moving averages from Ta-Lib, store functions in dictionary
talib_moving_averages = ['SMA', 'EMA', 'WMA', 'DEMA', 'KAMA', 'MIDPOINT', 'MIDPRICE', 'T3', 'TEMA', 'TRIMA']
talib_moving_averages = ['RSI']
functions = {}
for ma in talib_moving_averages:
    functions[ma] = abstract.Function(ma)
    print(functions[ma])
    functions[ma].parameters['timeperiod']=2
# Determine kurtosis "K" values for MA period 4-99
kurtosis_results = {'period': []}
for i in range(4, 60):
    kurtosis_results['period'].append(i)
    for ma in talib_moving_averages:
        
        # Run moving average, remove last 252 days (used later for test data set), trim MA result to last 60 days
        ma_output = functions[ma](data[:20], i).tail(100)
#         ma_output = functions[ma](data[:70], i)
        # Determine kurtosis "K" value
        k = kurtosis(ma_output, fisher=False)

        # add to dictionary
        if ma not in kurtosis_results.keys():
            kurtosis_results[ma] = []
        kurtosis_results[ma].append(k)

kurtosis_results = pd.DataFrame(kurtosis_results)
# display(kurtosis_results)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     t = functions["EMA"](data)
    t = data['average']
    features=pd.DataFrame()
    features['average_diff'] = data['average'].diff()
    display(features)
    t=features['average_diff']
#     t = functions["RSI"](data)
    t= t[~pd.isnull(t)]
    len_data = len(t)
    len_train = round(float(len_data)*(1-60/len_data))
    len_test = len_data-len_train
    test_data= data.tail(len_test)
    test_actual = t.tail(len_test).to_list()
    close_actual = data['close'].tail(len_test).to_list()
    print(len_data,len_train,len_test)
    prediction,prediction2, mse, rmse, mape  = get_arima(t, len_train, len_test)
    comp = list(map(list,zip(prediction,prediction2,test_actual,close_actual)))
#     for x in comp:print(x[0]-x[1])
    compdf = pd.DataFrame(comp)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02)

candlesticks = go.Candlestick(x=test_data.index,
                open=test_data['open'],
                high=test_data['high'],
                low=test_data['low'],
                close=test_data['close'])

p1 = go.Scatter(x=test_data.index, y=prediction, line=dict(color='orange', width=1),yaxis="y2")
p2 = go.Scatter(x=test_data.index, y=prediction2, line=dict(color='red', width=1),yaxis="y2")
avg = go.Scatter(x=test_data.index, y=test_actual, line=dict(color='black', width=1),yaxis="y2")
# lwma = go.Scatter(x=df.index, y=df.lwma, line=dict(color='green', width=1))

# macd = go.Scatter(x=df.index, y=df.macd, line=dict(color='blue', width=1),yaxis="y2")
# signal = go.Scatter(x=df.index, y=df.signal, line=dict(color='red', width=2),yaxis="y2")

# rsi = go.Scatter(x=df.index, y=df.rsi, line=dict(color='gray', width=2),yaxis="y3")

# data = [candlesticks,swma,lwma,macd,signal,rsi]
chart_full = [candlesticks,p1,p2,avg]
layout = go.Layout( yaxis=dict(
        domain=[.33, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[.2, .33]
    ),
    yaxis3=dict(
    domain=[0, .2]
    ),
)

fig = go.Figure(data=chart_full,layout=layout)
fig.show()

,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-09-04 04:00:00,347.00,347.00,346.86,346.89,81,346.989,4
2020-09-04 04:00:05,346.89,346.89,346.89,346.89,0,346.890,0
2020-09-04 04:00:10,346.89,346.89,346.89,346.89,0,346.890,0
2020-09-04 04:00:15,347.00,347.00,347.00,347.00,1,347.000,1
2020-09-04 04:00:20,347.00,347.00,347.00,347.00,0,347.000,0
...,...,...,...,...,...,...,...
2020-09-04 19:59:35,341.99,341.99,341.99,341.99,0,341.990,0
2020-09-04 19:59:40,342.00,342.00,342.00,342.00,1,342.000,1
2020-09-04 19:59:45,342.00,342.00,342.00,342.00,0,342.000,0


,date,open,high,low,close,volume,average
0,2020-09-04 12:00:00,339.22,339.22,339.11,339.12,78,339.137
1,2020-09-04 12:00:05,339.13,339.33,339.13,339.28,214,339.259
2,2020-09-04 12:00:10,339.27,339.29,339.21,339.27,76,339.241
3,2020-09-04 12:00:15,339.24,339.36,339.24,339.32,66,339.299
4,2020-09-04 12:00:20,339.35,339.37,339.29,339.31,54,339.337
...,...,...,...,...,...,...,...
716,2020-09-04 12:59:40,340.86,340.95,340.82,340.95,67,340.903
717,2020-09-04 12:59:45,340.99,340.99,340.87,340.89,31,340.934
718,2020-09-04 12:59:50,340.88,340.97,340.88,340.96,27,340.932
719,2020-09-04 12:59:55,340.95,340.95,340.87,340.91,22,340.915


RSI([input_arrays], [timeperiod=14])

Relative Strength Index (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 14
Outputs:
    real


,average_diff
0,NaN
1,0.122
2,-0.018
3,0.058
4,0.038
5,-0.098
6,-0.163
7,0.116
8,0.057
9,0.161


720 660 60
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-1591.197, Time=0.28 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-1562.867, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1593.120, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-1596.789, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-1595.193, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=-1595.198, Time=0.27 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-1593.184, Time=0.24 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-1595.163, Time=0.17 sec

Best model:  ARIMA(0,0,1)(0,0,0)[0]          
Total fit time: 1.221 seconds
ARIMA order: (0, 0, 1) 

working on 1 of 60 -- 1% complete
working on 2 of 60 -- 3% complete
working on 3 of 60 -- 5% complete
working on 4 of 60 -- 6% complete
working on 5 of 60 -- 8% complete
working on 6 of 60 -- 10% complete
working on 7 of 60 -- 11% complete
working on 8 of 60 -- 13% complete
working on 9 of 60 -- 15% complet

In [204]:

data = pd.read_csv('NVDA_bar_sample_5secs.csv', index_col=0, header=0)
display(data)
data = data.loc['2020-09-04 12:00:00':'2020-09-04 13:00:00'].reset_index(drop=False)
# display(data)
data['volume'] = data['volume'].astype('int32')
# del data["average"]
del data["barcount"]
def calcArima(data,column,len_test=10,diff = 1 ):
    try:del data['date']
    except: pass
    len_data = len(t)
    len_train = len_data-len_test
    test_starters = data.iloc[[len_train]]
#     display(data)
    test_data_orig = data.tail(len_test)
    if diff == 1:
        datad = data[column].diff()
        datad= datad.drop(data.index[0])
#         print(datad)
    test_data= datad.tail(len_test)
    train_data = datad.head(len_train)
#     display(train_data,test_data)
#     return train_data,test_data,test_starters

    start = test_starters[column].values[0]
#     print(start)
    train = train_data.values.tolist()
    test = test_data.values.tolist()
    test_len = len(test)
    # Initialize model
    model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    last_actual = start
    actuals = []
    prediction = []
    prediction_chain = []
    for i in range(len(test)):
#         print(test[i])
        model = pm.ARIMA(order=order)
        model.fit(train)
#         print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction_list = model.predict()
#         print(prediction_list)
        prediction_chain.append(prediction_list)
#         prediction.append(model.predict()[0])
#         print(prediction)
        train.append(test[i])
#         print(test[i])
        if diff == 1:
            prediction.append(last_actual+model.predict()[3])
            actuals.append(last_actual+test[i])
            last_actual = last_actual+test[i]
        else: actuals.append(test[i])
#         print(model.predict()[0],test[i],model.predict()[0]-test[i])
    # Generate error data
    mse = mean_squared_error(test, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    
    return test_data_orig,prediction,actuals,prediction_chain, mse, rmse, mape

# train_data,test_data,test_starters,undiff = splitData(data,len_test=10)
test_data,prediction,actuals,prediction_chain, mse, rmse, mape = calcArima(data,"average",len_test=30,diff =1)


,open,high,low,close,volume,average,barcount
date,,,,,,,
2020-09-04 04:00:00,512.00,512.00,512.00,512.00,20,512.00,2
2020-09-04 04:00:05,512.00,512.00,512.00,512.00,0,512.00,0
2020-09-04 04:00:10,512.00,512.00,512.00,512.00,0,512.00,0
2020-09-04 04:00:15,512.00,512.00,512.00,512.00,0,512.00,0
2020-09-04 04:00:20,512.00,512.00,512.00,512.00,0,512.00,0
...,...,...,...,...,...,...,...
2020-09-04 19:59:35,499.55,499.55,499.55,499.55,0,499.55,0
2020-09-04 19:59:40,499.55,499.55,499.55,499.55,0,499.55,0
2020-09-04 19:59:45,499.55,499.55,499.55,499.55,0,499.55,0


1      0.153
2      0.581
3      0.502
4      0.651
5     -0.206
       ...  
686    0.060
687   -0.323
688    0.043
689    0.148
690   -0.286
Name: average, Length: 690, dtype: float64

691   -0.163
692   -0.193
693    0.095
694    0.018
695   -0.706
696    0.023
697   -0.057
698    0.086
699   -0.143
700   -0.003
701    0.336
702    0.794
703    0.342
704    0.057
705   -0.101
706    0.162
707   -0.040
708   -0.405
709    0.329
710   -0.340
711    0.279
712    0.110
713    0.045
714   -0.421
715    0.396
716    0.172
717    0.265
718   -0.143
719   -0.065
720   -0.177
Name: average, dtype: float64

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=270.026, Time=0.38 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=290.351, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=265.574, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=265.229, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=267.202, Time=0.07 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=267.214, Time=0.07 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=269.032, Time=0.26 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=267.101, Time=0.18 sec

Best model:  ARIMA(0,0,1)(0,0,0)[0]          
Total fit time: 1.109 seconds
ARIMA order: (0, 0, 1) 



In [203]:
print(prediction_chain[0][0])

-0.058020820757228364


In [201]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02)

candlesticks = go.Candlestick(x=test_data.index,
                open=test_data['open'],
                high=test_data['high'],
                low=test_data['low'],
                close=test_data['close'])

p1 = go.Scatter(x=test_data.index, y=prediction, line=dict(color='orange', width=1))
avg = go.Scatter(x=test_data.index, y=actuals, line=dict(color='red', width=1))
# avg = go.Scatter(x=test_data.index, y=test_actual, line=dict(color='black', width=1),yaxis="y2")
# lwma = go.Scatter(x=df.index, y=df.lwma, line=dict(color='green', width=1))

# macd = go.Scatter(x=df.index, y=df.macd, line=dict(color='blue', width=1),yaxis="y2")
# signal = go.Scatter(x=df.index, y=df.signal, line=dict(color='red', width=2),yaxis="y2")

# rsi = go.Scatter(x=df.index, y=df.rsi, line=dict(color='gray', width=2),yaxis="y3")

# data = [candlesticks,swma,lwma,macd,signal,rsi]
chart_full = [candlesticks,p1,avg]
layout = go.Layout( yaxis=dict(
        domain=[.33, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[.2, .33]
    ),
    yaxis3=dict(
    domain=[0, .2]
    ),
)

fig = go.Figure(data=chart_full,layout=layout)
fig.show()


# fig = go.Figure(data=[go.Candlestick(x=test_data.index,
#                 open=test_data['open'], high=test_data['high'],
#                 low=test_data['low'], close=test_data['close'])
#                      ])

# fig.update_layout(xaxis_rangeslider_visible=False)
# fig.show()

In [33]:


optimized_period = {}
for ma in talib_moving_averages:
    difference = np.abs(kurtosis_results[ma] - 2)
    df = pd.DataFrame({'difference': difference, 'period': kurtosis_results['period']})
    df = df.sort_values(by=['difference'], ascending=True).reset_index(drop=True)
    if df.at[0, 'difference'] < 2 * 0.05:
        optimized_period[ma] = int(df.at[0, 'period'])
    else:
        print(ma + ' is not viable, best K greater or less than 3 +/-5%')

print('\nOptimized periods:', optimized_period)


Optimized periods: {'SMA': 4}


In [35]:


# Determine period with K closest to 3 +/-5%
optimized_period = {}
for ma in talib_moving_averages:
    difference = np.abs(kurtosis_results[ma] - 2)
    df = pd.DataFrame({'difference': difference, 'period': kurtosis_results['period']})
    df = df.sort_values(by=['difference'], ascending=True).reset_index(drop=True)
    if df.at[0, 'difference'] < 2 * 0.05:
        optimized_period[ma] = int(df.at[0, 'period'])
    else:
        print(ma + ' is not viable, best K greater or less than 3 +/-5%')

print('\nOptimized periods:', optimized_period)

simulation = {}
for ma in optimized_period:
    # Split data into low volatility and high volatility time series
    low_vol = functions[ma](data, optimized_period[ma])
    high_vol = data['close'] - low_vol

    # Generate ARIMA and LSTM predictions
    print('\nWorking on ' + ma + ' predictions')
    try:
        low_vol_prediction, low_vol_mse, low_vol_rmse, low_vol_mape = get_arima(low_vol, 1000, 70)
    except:
        print('ARIMA error, skipping to next MA type')
        continue
    high_vol_prediction, high_vol_mse, high_vol_rmse, high_vol_mape = get_lstm(high_vol, 1000, 70)

    final_prediction = pd.Series(low_vol_prediction) + pd.Series(high_vol_prediction)
    mse = mean_squared_error(final_prediction.values, data['close'].tail(70).values)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(data['close'].tail(70).reset_index(drop=True), final_prediction)

    # Generate prediction accuracy
    actual = data['close'].tail(70).values
    result_1 = []
    result_2 = []
    for i in range(1, len(final_prediction)):
        # Compare prediction to previous close price
        if final_prediction[i] > actual[i-1] and actual[i] > actual[i-1]:
            result_1.append(1)
        elif final_prediction[i] < actual[i-1] and actual[i] < actual[i-1]:
            result_1.append(1)
        else:
            result_1.append(0)

        # Compare prediction to previous prediction
        if final_prediction[i] > final_prediction[i-1] and actual[i] > actual[i-1]:
            result_2.append(1)
        elif final_prediction[i] < final_prediction[i-1] and actual[i] < actual[i-1]:
            result_2.append(1)
        else:
            result_2.append(0)

    accuracy_1 = np.mean(result_1)
    accuracy_2 = np.mean(result_2)

    simulation[ma] = {'low_vol': {'prediction': low_vol_prediction, 'mse': low_vol_mse,
                                  'rmse': low_vol_rmse, 'mape': low_vol_mape},
                      'high_vol': {'prediction': high_vol_prediction, 'mse': high_vol_mse,
                                   'rmse': high_vol_rmse},
                      'final': {'prediction': final_prediction.values.tolist(), 'mse': mse,
                                'rmse': rmse, 'mape': mape},
                      'accuracy': {'prediction vs close': accuracy_1, 'prediction vs prediction': accuracy_2}}



Optimized periods: {'SMA': 4}

Working on SMA predictions
ARIMA error, skipping to next MA type


In [24]:


# save simulation data here as checkpoint
with open('simulation_data.json', 'w') as fp:
    json.dump(simulation, fp)

for ma in simulation.keys():
    print('\n' + ma)
    print('Prediction vs Close:\t\t' + str(round(100*simulation[ma]['accuracy']['prediction vs close'], 2))
          + '% Accuracy')
    print('Prediction vs Prediction:\t' + str(round(100*simulation[ma]['accuracy']['prediction vs prediction'], 2))
          + '% Accuracy')
    print('MSE:\t', simulation[ma]['final']['mse'],
          '\nRMSE:\t', simulation[ma]['final']['rmse'],
          '\nMAPE:\t', simulation[ma]['final']['mape'])


SMA
Prediction vs Close:		49.4% Accuracy
Prediction vs Prediction:	47.01% Accuracy
MSE:	 0.08854614942168564 
RMSE:	 0.29756705029570335 
MAPE:	 0.05576032987504942

EMA
Prediction vs Close:		45.02% Accuracy
Prediction vs Prediction:	47.81% Accuracy
MSE:	 0.08249727822587008 
RMSE:	 0.28722339428721694 
MAPE:	 0.05411864618401112

WMA
Prediction vs Close:		44.62% Accuracy
Prediction vs Prediction:	49.0% Accuracy
MSE:	 0.08079693370778787 
RMSE:	 0.2842480144306867 
MAPE:	 0.0531345209073294

DEMA
Prediction vs Close:		45.02% Accuracy
Prediction vs Prediction:	49.0% Accuracy
MSE:	 0.08099735774065889 
RMSE:	 0.284600347400805 
MAPE:	 0.052372320048617436

KAMA
Prediction vs Close:		52.99% Accuracy
Prediction vs Prediction:	47.81% Accuracy
MSE:	 0.07916316475640153 
RMSE:	 0.28135949380890196 
MAPE:	 0.05184406648114709

MIDPOINT
Prediction vs Close:		52.59% Accuracy
Prediction vs Prediction:	48.21% Accuracy
MSE:	 0.08208808653171872 
RMSE:	 0.2865101857381666 
MAPE:	 0.0532380178026631